# <span style="color:#F26835; font-size:34px;" >Logistic Regression</span>
###### <span style="color:#A1A1A1;"> Students: Sigurður Baldursson, Þórhildur Þorleiksdóttir </span>
###### <span style="color:#A1A1A1;">Instructor: Magnús Eðvald Björnsson </span>

In [318]:
# Bokeh for plotting
from bokeh.plotting import figure, show, ColumnDataSource, output_notebook
from bokeh.models.tools import HoverTool, BoxZoomTool, CrosshairTool, Tool
import numpy as np
import pandas as pd
import statsmodels.api as sm

output_notebook()

Loading BokehJS ...

##  <span style="color:#498BA6;"> Logit function:</span>
A logit function is simply a function of the mean of the response variable Y that we use as the response instead of Y itself.

Because Y is a categorical binary variable and we need to predict in percentages, the logistic model uses the logit function to help us transform it into a response between 0 and 1

In the formula below: $\beta_{0} + \beta_{1}X_{1} $ for simple logisitic regression can be changed for multiple logisitic regression for the linear compination of independent variables $\beta_{0} + \beta_{1}X_{1} + \beta_{2}X_{2} + \dotso + \beta_{k}X_{k } $



 $ Ln  \bigg( \dfrac{{P}} {1 - P} \bigg) = \beta_{0} + \beta_{1}X_{1}  $ 




### <span style="color:#F2B33D;">Visualizing the logit:</span>
> $ logit(x) = Ln  \bigg( \dfrac{{x}} {1 - x} \bigg)  $

In [309]:
p_x = np.arange(0.0001,0.999999999,0.0001).tolist()

r_y = [np.log(i/(1-i)) for i in p_x]

def set_data_source(r_x,r_y):
    return ColumnDataSource(
        data=dict(
            x=r_x, 
            y=r_y,
        )
    )
    
def plot(l_x = [], l_y = [], s_x = [], s_y = [],
         width = 600,height = 400, 
         x_label = "", y_label = "", title = "", 
         line = False, scatter = False):

    plot = figure(plot_width=width, plot_height=height, 
              x_axis_label = x_label,
                y_axis_label = y_label,
              title=title,
            )
    if line:
        l_source = set_data_source(l_x,l_y)
        plot.line('x','y', source = l_source)
        
    if scatter:
        s_source = set_data_source(s_x,s_y)
        plot.circle('x', 'y', size=20, source = s_source)
        
    show(plot)

plot(p_x,r_y,[],[],400,400,"Percentages","", "Logit function: Ln(P/1-P)", True)


You can see it is reaching to -inf and inf as we get close to 0 and 1

Then we compute the inverse logit because we want our percentage as the Y variable / dependent. 

The inverse log of the the logit function allows us to find estimated regression equation.

$ \dfrac{{P}} {1 - P} = e^{\beta_{0} + \beta_{1}X_{1}}  $ 

$ P = e^{\beta_{0} + \beta_{1}X_{1} } \big( {1 - P} \big) $ 


... Until we end up with the estimated regression equation

$ \widehat{P} = \dfrac{e^{\beta_{0} + \beta_{1}X_{1}}} {1 + e^{\beta_{0} + \beta_{1}X_{1}} } $



### <span style="color:#F2B33D;">Visualizing the inverse logit:</span>
> $ logit^{-1}(a) = \dfrac{e^{a} } {1 + e^{a}}  $

> We can see below that the function forms a sigmoid curve or S curve



In [310]:
p_x = np.arange(-10,10,0.001).tolist()

r_y = [np.exp(i)/(1+np.exp(i)) for i in p_x]

plot(p_x,r_y,[],[],600,400,"","Percentages","Inverse Logit function: e(a)/1-e(a)", True)

## <span style="color:#B19B7D;"> A simple example calculated </span>

Lets say we are an individual applying for a loan in an Icelandic bank and
want to know the probability of our creditscore getting us a loan at the bank.
##### Data for each applicant:
* x-axis: Icelandic credit score points ranging 100(bad credit score) to 400(excellent)
* y-axis: Approved as 1 or 0

You can get your individual credit score from [Credit Info](https://www.creditinfo.is)

In [330]:
# Read the data in
data = pd.read_csv("loans.csv")

cols_to_keep = ["accepted",'creditscore']
df = data[cols_to_keep]
x_credit_scores = df.creditscore.values.tolist()
y_zeroes_and_ones = df.accepted.values.tolist()

sample_size = sum(df.accepted.value_counts())

plot([],[],x_credit_scores,y_zeroes_and_ones, 800, 400, 
     "Credit score points","Accepted","Credit Scores of " + str(sample_size) + " individuals on a scatter plot",
    False, True)

print("Our data has", sample_size, "rows")
print(df.accepted.value_counts())


Our data has 2500 rows
1    1282
0    1218
Name: accepted, dtype: int64


Next we want to plot the line but first we need to find out our coefficients.

The regression coefficients represent the change in the logit for each unit change in the predictor(i.e.indepenent variable)

The regression coefficients for logistic regression are calculated using maximum likelihood estimation or MLE. That is a topic of machine learning algorithms and a whole other lecture. If you want more info on MLE from
statsmodel you can call print(result.mle_settings). Our example reveales that MLE uses the Newton optimizer.


We can use the Logit function in statsmodel to get the coefficients for us.

Lets look at the data:

In [331]:
print(df.head(), '\n')

   accepted  creditscore
0         0          356
1         0          369
2         0          354
3         0          358
4         0          374 



Count accepted vs. not accepted

In [332]:
print(df.accepted.value_counts())

1    1282
0    1218
Name: accepted, dtype: int64


> ##### According to the statsmodel docs on the Logit method:
    An intercept is not included by default so it should be added by the user.

In [315]:
df['intercept'] = 1.00

train_cols = df.columns[1:3] 
# Index(['creditscore', 'intercept'], dtype='object')

# the Logit uses the Newton optimizer in Maximum Likelihood Estimation (MLE), 
# (see print(result.mle_settings) )

logit = sm.Logit(df['accepted'], df[train_cols])

result = logit.fit()
coef_arr = logit.fit().params.values
credit_coef, intercept_coef = np.split(coef_arr,2)

print(result.summary())

Optimization terminated successfully.
         Current function value: 0.351546
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.351546
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:               accepted   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      998
Method:                           MLE   Df Model:                            1
Date:                Sat, 25 Feb 2017   Pseudo R-squ.:                  0.4908
Time:                        21:36:38   Log-Likelihood:                -351.55
converged:                       True   LL-Null:                       -690.41
                                        LLR p-value:                2.090e-149
                  coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
cred

**From our output we get a <u>positive</u> coefficient slope for the credit score.**

**That means that the probability of getting a loan <u>increases</u> as we get a higher credit score.**

 ##### Now we can put the coefficients into the estimated regression equation:
$$ \widehat{P} = \dfrac{e^{\beta_{0} + \beta_{1}X_{1}}} {1 + e^{\beta_{0} + \beta_{1}X_{1}} } $$

 $$ X_{1} \text{is the value we want to test}$$

In [322]:
# Input we want to test
test_x = 230

def phat(test_x):
    a = intercept_coef[0] + credit_coef[0] * test_x
    return np.exp(a) / (1 + np.exp(a))

p_hat = phat(test_x)
print(p_hat,"\n")
print("Someone witha credit score of", test_x, "has an estimated probabilty of", "%0.2f" % (p_hat*100), "% to get a loan." )


0.0824132938373 

Someone witha credit score of 230 has an estimated probabilty of 8.24 % to get a loan.


In [317]:
line_x = [i for i in range(100,400)]
line_y = [phat(x) for x in range(100,400)]
plot(line_x,line_y, x_credit_scores,y_zeroes_and_ones,800, 400, 
     "Credit score","Percentages","Credit Scores of " + str(sample_size) + " individuals on a scatter plot",
    True, True)


## Multiple Logistic Regression:


## <span style="color:#498BA6;"> Appendix 1:</span>
### <span style="color:#F2B33D;">Generation code for loan data:</span>




In [328]:
#!/usr/bin/env python3
import sys
import os
import random
import math

# Constants
''' Parts is constant 5 because risk factor grades are in 5 parts'''
parts = 5

# Config Variables
'''Remove the loans.csv file'''
% rm loans.csv # Comment this line if you are not running this in jupyter notebook
               # or know what you are doing. Currently this supports partial data
                # in the form that you must have 0 and 1 at the front but rest must be empty

loan_info_file = "loans.csv"

lines_for_each_part = 500
#------

def is_not_zero_file(fpath):
	return os.path.isfile(fpath) and os.path.getsize(fpath) > 0

def read_file(ppl):
	with open(loan_info_file, 'r') as f:
		for l in f.read().split('\n')[1:]:
			l = l.strip()
			if not l:
				continue
			l = l.split(',')
			per = {}
			per['accepted'] = l[0]
			if len(l) > 1:
				per['creditscore'] = l[1]
				per['amount'] = l[2]
				per['age'] = l[3]
				per['marital'] = l[4]
            
			ppl.append(per)

def create_new(ppl):

	# Try to generate evenly distributed 0 and 1 for accepted/not accepted:
	'''
	To help us do that we will be using
	The Credit Info Risk Factor Grade that are defined as follows
	A = 353-400+, 
	B 317-353, 
	C = 281-317, 
	D = 235-281, 
	E = 100-235

	E and D applicants do not get a loan unless it is very small or dep on other factors.
	'''
	
	A,B,C,D,E = 0,1,2,3,4
	
	for i in range(0,parts):
		for j in range(math.ceil(i * lines_for_each_part),  math.ceil((i+1) * lines_for_each_part)):
			
			per = {}

			if i == A:
				if random.randint(0,15) == 1:
					# Maybe some random people dont get a loan in A
					per['accepted'] = 0
				else:
					per['accepted'] = 1
				per['creditscore'] = random.randint(353,400)
				per['riskf'] = 'A'
			elif i == B:
				if random.randint(0,7) == 1:
					# Maybe some random people dont get a loan in B
					per['accepted'] = 0
					
				else:
					per['accepted'] = 1
				
				per['creditscore'] = random.randint(317,353)
				per['riskf'] = 'B'

			elif i == C:
				per['accepted'] = random.randint(0,1)
				per['creditscore'] = random.randint(281,317)

				per['riskf'] = 'C'

			elif i == D:
				if random.randint(0,3) == 1:
					per['accepted'] = 1
				else:
					per['accepted'] = 0
				
				per['creditscore'] = random.randint(235,281)
				per['riskf'] = 'D'

			elif i == E:
				if random.randint(0,40) == 1:
					# Maybe some random people in E get a loan
					per['accepted'] = 1
				else:
					per['accepted'] = 0
				per['creditscore'] = random.randint(100,235)

				per['riskf'] = 'E'
			
			ppl.append(per)


def gen_age(per):
	if per['accepted'] == 1:
		return random.randint(18, 90)
	elif per['riskf'] == 'E':
		return random.randint(55,90)
	elif per['riskf'] == 'D':
		if random.randint(0,4) == 2:
			return random.randint(18,30)
		return random.randint(55,90)
	else:
		return random.randint(18, 90)

def gen_marital(per):
	if per['age'] > 78:
		return 0
	if per['accepted'] == 1:
		if random.randint(0,6) == 3:
			return 0
		else:
			return 1
	if per['accepted'] == 0:
		if random.randint(0,2) == 1:
			return 1
		else:
			return 0

def gen_cscore(per):
	'''We will be basing the credit score the risk factor 
	The Credit Info Risk Factor Grade that are defined as follows
	A = 353-400+ points, B 317-353 points, C = 281-317, D = 235-281, E = 100-235'''
	if['riskf'] == 'A':
		return random.randint(353,400)
	if['riskf'] == 'B':
		return random.randint(317,352)
	if['riskf'] == 'C':
		return random.randint(281,316)
	if['riskf'] == 'D':
		return random.randint(235,280)
	if['riskf'] == 'E':
		return random.randint(100,235)
def gen_amount(per):
	if per['accepted'] == 1:
		if per['age'] > 50 or per['age'] < 25:
			# Small loan half mil to 10 mil
			return random.randint(500000,15000000)
		if per['riskf'] == 'E':
			# Small loan
			return random.randint(100000,6000000)
		else:
			# Possibly bigger loan
			return random.randint(1000000,70000000)
	else:
		return random.randint(1000000,100000000)	

def add_to_file(ppl):
	with open(loan_info_file, 'w') as f:
		f.write('accepted,creditscore,amount,age,marital\n')
		# Here you can sort the file by other things if you want
		for per in sorted(ppl, key=lambda x: x['accepted']):
			f.write('%s,%s,%s,%s,%s\n' % (per['accepted'], per['creditscore'], per['amount'],per['age'],per['marital']))

def main():
    # This list will store a dict for each line
    ppl = []
    
    if is_not_zero_file(loan_info_file):
        read_file(ppl)
    
    if not ppl:
        create_new(ppl)
        
    for per in ppl:
        '''This is the order in which we will determine each factor'''
        if 'age' not in per:
            per['age'] = gen_age(per)
        if 'marital' not in per:
            per['marital'] = gen_marital(per)
        if 'creditscore' not in per:
            per['creditscore'] = gen_cscore()
        if 'amount' not in per:
            per['amount'] = gen_amount(per)
    
    add_to_file(ppl)
    print("Length of file", loan_info_file, "is:" , len(ppl))
    
main()

Length of file loans.csv is: 2500
